In [97]:
import os
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import label_binarize
from tqdm import tqdm
from sklearn.metrics import classification_report, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import json
import numpy as np


In [98]:
# Enable GPU and multi-core processing
def set_device_and_cores():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    torch.set_num_threads(os.cpu_count())
    print(f"Using device: {device}")
    print(f"Using {os.cpu_count()} CPU cores")
    return device

device = set_device_and_cores()

Using device: cuda
Using 16 CPU cores


In [99]:
class SpectrogramDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (str): Root directory containing the spectrogram images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.data = []

        # Traverse all directories and fetch images
        for main_dir in os.listdir(root_dir):
            main_path = os.path.join(root_dir, main_dir)
            if os.path.isdir(main_path):
                for sub_dir in os.listdir(main_path):
                    sub_path = os.path.join(main_path, sub_dir)
                    if os.path.isdir(sub_path):
                        for img_file in os.listdir(sub_path):
                            if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                                self.data.append((os.path.join(sub_path, img_file), f"{main_dir}_{sub_dir}"))

        # Debug: Check the number of images loaded
        print(f"Number of images loaded: {len(self.data)}")

        if len(self.data) == 0:
            raise ValueError(f"No images found in the specified directory structure: {root_dir}")

        # Map class names to indices
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(sorted(set([d[1] for d in self.data])))}

        # Debug: Check available classes
        print(f"Available classes: {list(self.class_to_idx.keys())}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.class_to_idx[class_name]

        if self.transform:
            image = self.transform(image)

        return image, label

In [100]:
# Function to create task-specific data loaders
def create_task_dataloaders(root_dir, num_tasks, num_classes_per_task, transform=None, batch_size=16):
    """
    Create data loaders for MAML tasks.

    Args:
        root_dir (str): Root directory containing the spectrogram images.
        num_tasks (int): Number of tasks to create.
        num_classes_per_task (int): Number of classes per task.
        transform (callable, optional): Transformations for data augmentation.
        batch_size (int): Batch size for task-specific data loaders.

    Returns:
        list: A list of task-specific DataLoader pairs [(train_loader, val_loader), ...].
    """
    dataset = SpectrogramDataset(root_dir, transform=transform)
    tasks = []

    for _ in range(num_tasks):
        # Randomly select classes for the task
        selected_classes = random.sample(list(dataset.class_to_idx.keys()), k=num_classes_per_task)
        task_data = [(img, label) for img, label in dataset.data if label in selected_classes]

        # Create a dataset and split into train and val
        class_to_idx = {cls: idx for idx, cls in enumerate(selected_classes)}
        task_dataset = [(img, class_to_idx[label]) for img, label in task_data]
        train_size = int(0.8 * len(task_dataset))
        val_size = len(task_dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(task_dataset, [train_size, val_size])

        # Create DataLoaders
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count())
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count())

        tasks.append((train_loader, val_loader))

    return tasks

In [101]:
def maml_train(model, dataloaders, meta_optimizer, criterion, num_tasks, num_inner_steps, inner_lr, num_epochs=100):
    """
    Train a model using MAML.

    Args:
        model (torch.nn.Module): The model to train.
        dataloaders (list): List of task-specific DataLoader pairs [(train_loader, val_loader), ...].
        meta_optimizer (torch.optim.Optimizer): Optimizer for the meta-parameters.
        criterion (torch.nn.Module): Loss function.
        num_tasks (int): Number of tasks to sample per meta-update.
        num_inner_steps (int): Number of gradient steps in the inner loop.
        inner_lr (float): Learning rate for the inner loop.
        num_epochs (int): Number of meta-epochs.

    Returns:
        dict: Metrics tracked during training.
    """
    metrics = {
        'loss': [],
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1_score': [],
        'roc_auc': []
    }

    for epoch in range(num_epochs):
        meta_loss = 0.0
        meta_accuracy = 0.0
        meta_precision = 0.0
        meta_recall = 0.0
        meta_f1 = 0.0
        meta_roc_auc = 0.0

        for task_idx, (train_loader, val_loader) in enumerate(random.sample(dataloaders, num_tasks)):
            task_model = torch.nn.Module(model)  # Clone the model
            task_model.load_state_dict(model.state_dict())

            # Inner loop: Task-specific adaptation
            task_optimizer = torch.optim.SGD(task_model.parameters(), lr=inner_lr)
            for inner_step, (inputs, labels) in enumerate(train_loader):
                inputs, labels = inputs.to(device), labels.to(device)
                task_optimizer.zero_grad()
                outputs = task_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                task_optimizer.step()

            # Outer loop: Meta-optimization
            all_preds = []
            all_labels = []
            for val_inputs, val_labels in val_loader:
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_outputs = task_model(val_inputs)
                loss = criterion(val_outputs, val_labels)
                meta_loss += loss.item()

                _, preds = torch.max(val_outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(val_labels.cpu().numpy())

            # Calculate metrics for this task
            meta_accuracy += accuracy_score(all_labels, all_preds)
            meta_precision += precision_score(all_labels, all_preds, average='weighted', zero_division=0)
            meta_recall += recall_score(all_labels, all_preds, average='weighted', zero_division=0)
            meta_f1 += f1_score(all_labels, all_preds, average='weighted', zero_division=0)
            if len(set(all_labels)) > 1:
                meta_roc_auc += roc_auc_score(
                    label_binarize(all_labels, classes=list(range(len(set(all_labels))))),
                    torch.nn.functional.softmax(torch.tensor(all_preds), dim=0).numpy(),
                    average='weighted', multi_class='ovo'
                )

        # Update meta-parameters
        meta_optimizer.zero_grad()
        meta_loss /= len(dataloaders)
        meta_loss.backward()
        meta_optimizer.step()

        # Store metrics
        metrics['loss'].append(meta_loss)
        metrics['accuracy'].append(meta_accuracy / num_tasks)
        metrics['precision'].append(meta_precision / num_tasks)
        metrics['recall'].append(meta_recall / num_tasks)
        metrics['f1_score'].append(meta_f1 / num_tasks)
        if meta_roc_auc > 0:
            metrics['roc_auc'].append(meta_roc_auc / num_tasks)

        print(f"Epoch {epoch + 1}/{num_epochs} - Meta Loss: {meta_loss:.4f}, Accuracy: {meta_accuracy / num_tasks:.4f}, Precision: {meta_precision / num_tasks:.4f}, Recall: {meta_recall / num_tasks:.4f}, F1: {meta_f1 / num_tasks:.4f}, ROC AUC: {meta_roc_auc / num_tasks if meta_roc_auc > 0 else 0.0:.4f}")

    return metrics

In [102]:
# Generate plots for metrics
def plot_metrics(metrics, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for metric, values in metrics.items():
        plt.figure()
        plt.plot(values, label=metric)
        plt.xlabel('Epochs')
        plt.ylabel(metric.capitalize())
        plt.title(f'{metric.capitalize()} Over Epochs')
        plt.legend()
        plt.savefig(os.path.join(output_dir, f'{metric}.png'))
        plt.close()

In [103]:
# Save model summary
def save_model_summary(model, input_size, plot_dir):
    from torchsummary import summary
    os.makedirs(plot_dir, exist_ok=True)
    summary_str = []
    def print_fn(line):
        summary_str.append(line)
    summary(model, input_size=input_size, print_fn=print_fn, device=device)
    summary_path = os.path.join(plot_dir, "model_summary.txt")
    with open(summary_path, "w") as f:
        f.write("\n".join(summary_str))
    print(f"Model summary saved to {summary_path}")

In [104]:
if __name__ == "__main__":
    root_dir = "project/data/spectrograms"
    batch_size = 32
    val_split = 0.2

    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    # Create dataloaders for the entire dataset
    dataloaders = create_full_dataloaders(root_dir, transform=transform, batch_size=batch_size, val_split=val_split)

    # Define model, criterion, and optimizer
    num_classes = len(SpectrogramDataset(root_dir).class_to_idx)
    class SimpleCNN(torch.nn.Module):
        def __init__(self, num_classes):
            super(SimpleCNN, self).__init__()
            self.conv_layers = torch.nn.Sequential(
                torch.nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2, stride=2),
                torch.nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2, stride=2),
            )
            self.fc_layers = torch.nn.Sequential(
                torch.nn.Linear(32 * 32 * 32, 128),
                torch.nn.ReLU(),
                torch.nn.Linear(128, num_classes),
            )

        def forward(self, x):
            x = self.conv_layers(x)
            x = x.view(x.size(0), -1)  # Flatten
            x = self.fc_layers(x)
            return x

    model = SimpleCNN(num_classes=num_classes).to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # Train the model
    metrics, report = train_model_with_metrics(model, dataloaders, criterion, optimizer)

    # Plot metrics and save model summary
    plot_metrics(metrics, "plots/maml")
    save_model_summary(model, (3, 128, 128), "plots/maml")

Number of images loaded: 43529
Available classes: ['emergency_Alarm', 'emergency_Battle cry', 'emergency_Explosion', 'emergency_Firecracker', 'emergency_Gasp', 'emergency_Gunshot', 'emergency_Machine gun', 'emergency_Motor vehicle (road)', 'emergency_Reversing beeps', 'emergency_Scrape', 'emergency_Screaming', 'emergency_Tire squeal', 'emergency_Train horn', 'emergency_Vehicle horn', 'emergency_Whip', 'normal_Bird vocalization', 'normal_Breathing', 'normal_Child speech', 'normal_Female speech', 'normal_Human voice', 'normal_Laughter', 'normal_Male speech', 'normal_Mechanisms', 'normal_Music', 'normal_Sound effect', 'normal_Surface contact', 'normal_Tap', 'normal_Walk', 'normal_Wind']
Number of images loaded: 43529
Available classes: ['emergency_Alarm', 'emergency_Battle cry', 'emergency_Explosion', 'emergency_Firecracker', 'emergency_Gasp', 'emergency_Gunshot', 'emergency_Machine gun', 'emergency_Motor vehicle (road)', 'emergency_Reversing beeps', 'emergency_Scrape', 'emergency_Screami

Epoch 1/100: 100%|██████████| 1089/1089 [00:47<00:00, 22.71it/s]


Epoch 1/100 - Loss: 2.9639, Accuracy: 0.1380, Precision: 0.1289, Recall: 0.1380, F1 Score: 0.1260, ROC AUC: 0.7346


Epoch 2/100: 100%|██████████| 1089/1089 [00:35<00:00, 30.97it/s]


Epoch 2/100 - Loss: 2.5951, Accuracy: 0.2229, Precision: 0.2078, Recall: 0.2229, F1 Score: 0.2099, ROC AUC: 0.7812


...

Epoch 73/100: 100%|██████████| 1089/1089 [01:44<00:00, 10.41it/s]


Epoch 73/100 - Loss: 0.2164, Accuracy: 0.9373, Precision: 0.9571, Recall: 0.9373, F1 Score: 0.9435, ROC AUC: 0.9926


Epoch 74/100: 100%|██████████| 1089/1089 [01:50<00:00,  9.85it/s]


Epoch 74/100 - Loss: 0.2094, Accuracy: 0.9397, Precision: 0.9636, Recall: 0.9397, F1 Score: 0.9473, ROC AUC: 0.9927
Early stopping triggered.


ValueError: Found input variables with inconsistent numbers of samples: [2576976, 34824]

SyntaxError: unterminated string literal (detected at line 1) (981200306.py, line 1)